In [ ]:
# 读取市场状态数据
market_states = {}
for sector in sectors:
    try:
        market_state_file = f"results/sector{sector}/df_market_states.csv"
        if os.path.exists(market_state_file):
            df_market_state = pd.read_csv(market_state_file, index_col=0)
            for idx, row in df_market_state.iterrows():
                if idx not in market_states:
                    market_states[idx] = row.to_dict()
    except:
        pass

# 在投资组合优化中考虑市场状态
for i in range(len(trade_date)):
    # 获取当前市场状态
    current_market_state = None
    if trade_date[i] in market_states:
        current_market_state = market_states[trade_date[i]]['state']
        
    # 根据市场状态调整优化参数
    risk_free_rate = 0.01  # 默认无风险利率
    risk_aversion = 1.0    # 默认风险厌恶系数
    
    if current_market_state == "Bull":
        # 牛市中降低风险厌恶
        risk_aversion = 0.8
    elif current_market_state == "Bear":
        # 熊市中提高风险厌恶
        risk_aversion = 2.0
    elif current_market_state == "Sideways":
        # 盘整市场维持中性
        risk_aversion = 1.2
    
    # 获取选定股票的价格和预测收益...
    
    # 市场状态感知的优化
    if current_market_state:
        print(f"Current market state: {current_market_state}")
        
        # 根据市场状态调整协方差矩阵 - 熊市中更谨慎估计风险
        if current_market_state == "Bear":
            # 使用更保守的协方差估计，如Ledoit-Wolf协方差收缩
            S = risk_models.CovarianceShrinkage(p1_return_table_pivot).ledoit_wolf()
        else:
            # 使用样本协方差
            S = risk_models.sample_cov(p1_return_table_pivot)
        
        # 根据市场状态调整优化目标
        if current_market_state == "Bull":
            # 牛市更关注回报最大化
            ef_mean = EfficientFrontier(mu, S, weight_bounds=(0, 0.05))
            raw_weights_mean = ef_mean.max_sharpe(risk_free_rate=risk_free_rate)
        elif current_market_state == "Bear":
            # 熊市更关注风险最小化，使用风险平价方法
            ef_mean = EfficientFrontier(mu, S, weight_bounds=(0, 0.05))
            ef_mean.add_objective(objective_functions.L2_reg, gamma=risk_aversion)
            raw_weights_mean = ef_mean.min_volatility()
        else:
            # 震荡市使用最大Sharpe比率，但增加正则化
            ef_mean = EfficientFrontier(mu, S, weight_bounds=(0, 0.05))
            ef_mean.add_objective(objective_functions.L2_reg, gamma=0.5)
            raw_weights_mean = ef_mean.max_sharpe(risk_free_rate=risk_free_rate)
    else:
        # 原有优化代码...